## FCOM Rudics Prawler Parser -> CSV for erddap

- 2019 Arctic Mooring

In [9]:
import os
import glob #python >= 3.5

import datetime

import pandas as pd

import seawater as sw
import calc.aanderaa_corrO2_sal as optode_O2_corr

In [10]:
#list id's in download path
root_path = '/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001'

id_dir = os.listdir(root_path)
id_dir = sorted(id_dir)


In [11]:
profile_counter = 0 
# 19ckitaepr-2a 
#0,22
#  8:1
month='8'
day='1'

for indfile in sorted(glob.glob(root_path+'/CF001_0['+month+']_['+day+']*')):
    print(indfile)
    with open(indfile) as f:
        for line in f:
            if ('%%' in line):
                flag = line.split()[0]
                if ('PRAWC_F001' in line):
                    profile_counter += 1
                next(f)
                continue
            if not (line.strip()):
                continue
            if 'PRAWE_F001' in flag:
                with open("eng.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'GPS_F001' in flag:
                with open("gps.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')

            if 'PRAWC_F001' in flag:
                with open("PRAWC_F001.csv", "a") as myfile:
                    myfile.write(str(profile_counter) + ','+line.strip()+'\n')        
 
                    
print("Total Profile number:{profile_counter}".format(profile_counter=profile_counter))

/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_10_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_11_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_12_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_13_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_14_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_15_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_16_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/TELO/F001/CF001_08_17_2019
/Users/bell/ecoraid/2019/Moorings/19ckitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/T

In [12]:
line

'2019-08-19T23:18:22Z,004,D,10,08,15,005,031,250,09000,00196,00047,0448,0000,0,0,0,1\n'

In [13]:
FCOM001 = pd.read_csv('PRAWC_F001.csv',header=None,
                  names=['profilenumber','ep','CD','CT','CC','OT','02','CH','TB'])

FCOM001['datetime'] = pd.to_datetime([int(x,16) for x in FCOM001.ep], unit='s')
FCOM001.set_index(pd.DatetimeIndex(FCOM001['datetime']),inplace=True)
FCOM001 = FCOM001.drop(['datetime'], axis=1)


In [14]:
#redeployment of instrument
FCOM001.CD = FCOM001.CD
FCOM001.CT = FCOM001.CT
FCOM001.CC = FCOM001.CC
FCOM001.OT = FCOM001.OT
FCOM001['02'] = FCOM001['02']
#---
FCOM001.CH = (FCOM001.CH-60)*0.0226
#FCOM001.CH = (FCOM001.CH)
FCOM001.TB = (FCOM001.TB-51)*0.2022
#FCOM001.TB = FCOM001.TB
#---
# convert cond to salinity
FCOM001['S'] = sw.salt(r=FCOM001.CC/4.2914,t=FCOM001.CT,p=FCOM001.CD)

### sal correct and convert oxygen
FCOM001['O2_corr'] = optode_O2_corr.O2_sal_comp(oxygen_conc=FCOM001['02'],
                                     salinity=FCOM001.S,
                                     temperature=FCOM001.CT)
FCOM001['O2_corr_umkg'] = optode_O2_corr.O2_molar2umkg(oxygen_conc=FCOM001.O2_corr,
                                     salinity=FCOM001.S,
                                     temperature=FCOM001.CT,
                                     pressure=FCOM001.CD)

FCOM001['O2PercentSat'] = optode_O2_corr.O2PercentSat(oxygen_conc=FCOM001.O2_corr_umkg,
                                     salinity=FCOM001.S,
                                     temperature=FCOM001.CT,
                                     pressure=FCOM001.CD)

FCOM001['dep_lat']=71.2167
FCOM001['dep_lon']=-164.25

In [15]:
current_date = datetime.date.today().strftime('%Y%m%d')

FCOM001.to_csv('19CKITAEPR2A_FCOM001_20190'+month+'.'+day+'.csv', 
           date_format='%Y-%m-%d %H:%M:%S')